In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
import cv2
from PIL import Image

In [10]:
input_size = 28
num_classes = 10
num_epochs = 20
batch_size = 500

data_transforms = {
        'train': torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
            , torchvision.transforms.RandomCrop(32, padding=4)  # 先四周填充0，在吧图像随机裁剪成32*32
            , torchvision.transforms.RandomHorizontalFlip(p=0.5)  # 随机水平翻转 选择一个概率概率
            , torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 均值，标准差
        ]),
        'valid': torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
            , torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

train_dataset = torchvision.datasets.CIFAR10(root='data',
                              train=True,
                              transform=data_transforms['train'])

test_dataset = torchvision.datasets.CIFAR10(root='data',
                             train=False,
                             transform=data_transforms['valid'])

In [11]:
#构建 mini-batch
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [12]:
#定义准确率作为评估标准
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]            #torch.max()[0]返回最大值 torch.max()[1]返回最大值位置，既预测值
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)                          #这里返回的是正确个数与总个数组成的元组

In [1]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():

    return ResNet(ResidualBlock)

NameError: name 'nn' is not defined

In [34]:
net = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.cuda()
test_right = []
train_loss = []

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 50 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.cuda(), target.cuda()
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            test_right.append((100*val_r[0]/val_r[1]).cpu().numpy())
            train_loss.append(loss.data.cpu().numpy())

            print(f'当前epoch：{epoch} [{batch_idx*batch_size}/{len(train_loader.dataset)}]   损失：{loss.data:.6f}  训练集准确率：{100*train_r[0]/train_r[1]:.2f}%   测试集准确率：{100*val_r[0]/val_r[1]:.2f}%')

当前epoch：0 [0/50000]   损失：2.394521  训练集准确率：13.50%   测试集准确率：11.41%
当前epoch：0 [20000/50000]   损失：1.473894  训练集准确率：35.65%   测试集准确率：39.02%
当前epoch：0 [40000/50000]   损失：1.275994  训练集准确率：42.58%   测试集准确率：50.19%
当前epoch：1 [0/50000]   损失：1.172294  训练集准确率：56.50%   测试集准确率：50.43%
当前epoch：1 [20000/50000]   损失：0.999227  训练集准确率：61.82%   测试集准确率：59.23%
当前epoch：1 [40000/50000]   损失：0.914390  训练集准确率：63.68%   测试集准确率：63.22%
当前epoch：2 [0/50000]   损失：0.783401  训练集准确率：72.50%   测试集准确率：49.70%
当前epoch：2 [20000/50000]   损失：0.680087  训练集准确率：73.34%   测试集准确率：64.01%
当前epoch：2 [40000/50000]   损失：0.816586  训练集准确率：73.98%   测试集准确率：66.11%
当前epoch：3 [0/50000]   损失：0.547403  训练集准确率：81.00%   测试集准确率：71.57%
当前epoch：3 [20000/50000]   损失：0.492232  训练集准确率：80.33%   测试集准确率：73.34%
当前epoch：3 [40000/50000]   损失：0.488866  训练集准确率：80.28%   测试集准确率：76.01%
当前epoch：4 [0/50000]   损失：0.431932  训练集准确率：84.50%   测试集准确率：76.70%
当前epoch：4 [20000/50000]   损失：0.439816  训练集准确率：84.58%   测试集准确率：74.67%
当前epoch：4 [40000/50000]   损失：0.479009  训练集准确率：84.56%  

In [13]:
#构建cnn
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                             out_channels=32,
                                             kernel_size=5,
                                             stride=1,
                                             padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.out = nn.Sequential(nn.Flatten(),
                                 nn.Linear(64 * 8 * 8, 256),
                                 nn.ReLU(),
                                 nn.Linear(256, 10),)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        output = self.out(x)
        return output

In [14]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.cuda()
test_right = []
train_loss = []

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 50 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.cuda(), target.cuda()
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            test_right.append((100*val_r[0]/val_r[1]).cpu().numpy())
            train_loss.append(loss.data.cpu().numpy())

            print(f'当前epoch：{epoch} [{batch_idx*batch_size}/{len(train_loader.dataset)}]   损失：{loss.data:.6f}  训练集准确率：{100*train_r[0]/train_r[1]:.2f}%   测试集准确率：{100*val_r[0]/val_r[1]:.2f}%')

当前epoch：0 [0/50000]   损失：2.300746  训练集准确率：10.20%   测试集准确率：19.05%
当前epoch：0 [25000/50000]   损失：1.566785  训练集准确率：33.63%   测试集准确率：45.66%
当前epoch：1 [0/50000]   损失：1.412195  训练集准确率：47.60%   测试集准确率：50.58%
当前epoch：1 [25000/50000]   损失：1.294514  训练集准确率：50.05%   测试集准确率：55.64%
当前epoch：2 [0/50000]   损失：1.304163  训练集准确率：53.60%   测试集准确率：58.87%
当前epoch：2 [25000/50000]   损失：1.230042  训练集准确率：56.93%   测试集准确率：60.32%
当前epoch：3 [0/50000]   损失：1.069164  训练集准确率：60.80%   测试集准确率：62.66%
当前epoch：3 [25000/50000]   损失：1.076782  训练集准确率：61.24%   测试集准确率：65.43%
当前epoch：4 [0/50000]   损失：1.043490  训练集准确率：63.80%   测试集准确率：66.14%
当前epoch：4 [25000/50000]   损失：1.006514  训练集准确率：64.18%   测试集准确率：65.96%
当前epoch：5 [0/50000]   损失：0.933830  训练集准确率：69.20%   测试集准确率：67.88%
当前epoch：5 [25000/50000]   损失：0.998918  训练集准确率：65.71%   测试集准确率：69.45%
当前epoch：6 [0/50000]   损失：0.980122  训练集准确率：65.00%   测试集准确率：70.26%
当前epoch：6 [25000/50000]   损失：0.881896  训练集准确率：68.00%   测试集准确率：69.67%
当前epoch：7 [0/50000]   损失：0.783719  训练集准确率：73.40%   测试集准确率：69.6